Load dataframes with acf and calculate tau

In [1]:
import pandas as pd
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

In [2]:
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms\\'

### Functions

In [3]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    acf_2d = acf_df_[acf_cols_].values
    acf_2d = acf_2d.astype(np.float64) # RuntimeWarning: overflow encountered in matmul sttc start_idx 2
    n_units = acf_2d.shape[0]
    print('Calculating taus for {}'.format(acf_2d.shape))
    
    fit_popt_a_l, fit_popt_b_l, fit_popt_c_l = [],[],[]
    fit_tau_l = []
    fit_r_squared_l = []
    fit_log_message_l = []
    
    for i in range(n_units):
        fit_popt, fit_pcov, tau, fit_r_squared, log_message = fit_single_exp(acf_2d[i,:], start_idx_, func_single_exp_monkey)
        if  type(fit_popt) == np.ndarray:
            fit_popt_a_l.append(fit_popt[0])
            fit_popt_b_l.append(fit_popt[1])
            fit_popt_c_l.append(fit_popt[2])
        else:
            fit_popt_a_l.append(np.nan)
            fit_popt_b_l.append(np.nan)
            fit_popt_c_l.append(np.nan)
        fit_tau_l.append(tau)
        fit_r_squared_l.append(fit_r_squared)
        fit_log_message_l.append(log_message)
    
    data_df = np.vstack((fit_popt_a_l, fit_popt_b_l, fit_popt_c_l, fit_tau_l, fit_r_squared_l, fit_log_message_l)).T
    tau_df = pd.DataFrame(data_df, columns=['fit_a', 'fit_b', 'fit_c','tau', 'r_squared', 'log_message'])
    tau_df.insert(0, 'unit_id', acf_df_['unit_id'].values)

    for col in ['fit_a', 'fit_b', 'fit_c','tau', 'r_squared']:
        tau_df[col] = tau_df[col].astype(float) 
    
    return tau_df

### Calculate tau

#### One tau per area

2 methods of doing that:
* average ACF over units and fit
* fit using all ACFs from all neurons

In [4]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [5]:
area = 'pfdl' 

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d_mean = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_pearsonr_trial_avg_fit_popt, _, pfdl_pearsonr_trial_avg_tau, pfdl_pearsonr_trial_avg_fit_r_squared, _ = fit_single_exp(pearsonr_trial_avg_2d_mean, 
                                                                            start_idx_=2, exp_fun_=func_single_exp_monkey)
pfdl_pearsonr_trial_avg_tau_ms = pfdl_pearsonr_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_pearsonr_trial_avg_tau_ms, 
                                                             pfdl_pearsonr_trial_avg_fit_popt, 
                                                             pfdl_pearsonr_trial_avg_fit_r_squared))

sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_avg_2d_mean = np.nanmean(sttc_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_sttc_trial_avg_fit_popt, _, pfdl_sttc_trial_avg_tau, pfdl_sttc_trial_avg_fit_r_squared, _ = fit_single_exp(sttc_trial_avg_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfdl_sttc_trial_avg_tau_ms = pfdl_sttc_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_sttc_trial_avg_tau_ms, 
                                                             pfdl_sttc_trial_avg_fit_popt, 
                                                             pfdl_sttc_trial_avg_fit_r_squared))

sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_concat_2d_mean = np.nanmean(sttc_trial_concat_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfdl_sttc_trial_concat_fit_popt, _, pfdl_sttc_trial_concat_tau, pfdl_sttc_trial_concat_fit_r_squared, _ = fit_single_exp(sttc_trial_concat_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfdl_sttc_trial_concat_tau_ms = pfdl_sttc_trial_concat_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfdl_sttc_trial_concat_tau_ms, 
                                                             pfdl_sttc_trial_concat_fit_popt, 
                                                             pfdl_sttc_trial_concat_fit_r_squared))

tau 240.75047875443357, popt [0.21563292 0.20768391 0.1519869 ], fit_r_squared 0.9954832781393308
tau 243.25030236826842, popt [0.17114559 0.20554959 0.56114312], fit_r_squared 0.9962669955022356
tau 179.2073997377251, popt [0.31666675 0.27900634 0.19925986], fit_r_squared 0.9760184400425521


In [6]:
area = 'pfp' 

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d_mean = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_pearsonr_trial_avg_fit_popt, _, pfp_pearsonr_trial_avg_tau, pfp_pearsonr_trial_avg_fit_r_squared, _ = fit_single_exp(pearsonr_trial_avg_2d_mean, 
                                                                            start_idx_=2, exp_fun_=func_single_exp_monkey)
pfp_pearsonr_trial_avg_tau_ms = pfp_pearsonr_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_pearsonr_trial_avg_tau_ms, 
                                                             pfp_pearsonr_trial_avg_fit_popt, 
                                                             pfp_pearsonr_trial_avg_fit_r_squared))

sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_avg_2d_mean = np.nanmean(sttc_trial_avg_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_sttc_trial_avg_fit_popt, _, pfp_sttc_trial_avg_tau, pfp_sttc_trial_avg_fit_r_squared, _ = fit_single_exp(sttc_trial_avg_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfp_sttc_trial_avg_tau_ms = pfp_sttc_trial_avg_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_sttc_trial_avg_tau_ms, 
                                                             pfp_sttc_trial_avg_fit_popt, 
                                                             pfp_sttc_trial_avg_fit_r_squared))

sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
sttc_trial_concat_2d_mean = np.nanmean(sttc_trial_concat_df[acf_cols].values, axis=0) # todo remove all rows with NaNs? 
pfp_sttc_trial_concat_fit_popt, _, pfp_sttc_trial_concat_tau, pfp_sttc_trial_concat_fit_r_squared, _ = fit_single_exp(sttc_trial_concat_2d_mean, 
                                                                                                                start_idx_=2, 
                                                                                                                exp_fun_=func_single_exp_monkey)
pfp_sttc_trial_concat_tau_ms = pfp_sttc_trial_concat_tau*bin_size
print('tau {}, popt {}, fit_r_squared {}'.format(pfp_sttc_trial_concat_tau_ms, 
                                                             pfp_sttc_trial_concat_fit_popt, 
                                                             pfp_sttc_trial_concat_fit_r_squared))

tau 223.54058813497463, popt [0.17090143 0.22367303 0.21222378], fit_r_squared 0.992286742303797
tau 222.75020201159091, popt [0.13212734 0.22446669 0.6686685 ], fit_r_squared 0.9893918617881063
tau 198.14650064517286, popt [0.22805424 0.25233855 0.25476348], fit_r_squared 0.9717247849009442


#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [ ]:
area = 'pfp' # pfp

n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

### todo (metrics)

In [ ]:
n_rows_with_nans = pearsonr_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(pearsonr_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_pearsonr_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_pearsonr_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))

In [ ]:
n_rows_with_nans = sttc_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(sttc_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_sttc_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_sttc_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))